# Genearting Continuations

### Import the libraries

In [1]:
import os
import pickle
import random
import tqdm
import json

import miditok
from music21 import converter, meter, stream, key

from torch.cuda.amp import autocast, GradScaler # Mixed Precision
import torch
from torch.utils.data import DataLoader, Dataset
from x_transformer2 import TransformerWrapper, Decoder, AutoregressiveWrapper
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Load the model

In [2]:
dim_model = 1024
depth_model = 32
max_sequence = 1024

model = TransformerWrapper(
    num_tokens = 2002,
    max_seq_len = max_sequence,
    attn_layers = Decoder(dim = dim_model, depth = depth_model, heads = 16)
)

model = AutoregressiveWrapper(model)
model = torch.nn.DataParallel(model)
model = model.to(device)

In [3]:
checkpoint = torch.load("ckpt/good_bpe/latest_bpe.pth", map_location='cpu')
model.load_state_dict(checkpoint['state_dict'])
#optim.load_state_dict(checkpoint['optimizer'])

<All keys matched successfully>

In [4]:
model.eval()

DataParallel(
  (module): AutoregressiveWrapper(
    (net): TransformerWrapper(
      (token_emb): TokenEmbedding(
        (emb): Embedding(2002, 1024)
      )
      (pos_emb): AbsolutePositionalEmbedding(
        (emb): Embedding(1024, 1024)
      )
      (post_emb_norm): Identity()
      (emb_dropout): Dropout(p=0.0, inplace=False)
      (project_emb): Identity()
      (attn_layers): Decoder(
        (layers): ModuleList(
          (0): ModuleList(
            (0): ModuleList(
              (0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
              (1-2): 2 x None
            )
            (1): Attention(
              (to_q): Linear(in_features=1024, out_features=1024, bias=False)
              (to_k): Linear(in_features=1024, out_features=1024, bias=False)
              (to_v): Linear(in_features=1024, out_features=1024, bias=False)
              (attend): Attend(
                (attn_dropout): Dropout(p=0.0, inplace=False)
              )
              (to_out): Li

### Setup the MIDI tokenizer

In [5]:
# Our parameters
CHORD_MAPS = {
    "min": (0, 3, 7),
    "maj": (0, 4, 7),
    "dim": (0, 3, 6),
    "aug": (0, 4, 8),
    "sus2": (0, 2, 7),
    "sus4": (0, 5, 7),
    "7dom": (0, 4, 7, 10),
    "7min": (0, 3, 7, 10),
    "7maj": (0, 4, 7, 11),
    "7halfdim": (0, 3, 6, 10),
    "7dim": (0, 3, 6, 9),
    "7aug": (0, 4, 8, 11),
    "9maj": (0, 4, 7, 10, 14),
    "9min": (0, 4, 7, 10, 13),
}

TS_MAPS = {16: [15, 11, 7, 4, 3, 12, 6], 8: [3, 12, 6, 7, 5, 9], 4: [5, 6, 3, 2, 1, 4, 9], 2: [3, 2, 4, 1]}


TOKENIZER_PARAMS2 = {
    "pitch_range": (21, 109),
    "beat_res": {(0, 4): 8, (4, 12): 4},
    "nb_velocities": 32,
    "special_tokens": ["PAD", "EOS"],
    "use_chords": True,
    "chord_maps": CHORD_MAPS,
    "use_rests": True,
    "use_tempos": True,
    "use_time_signatures": True,
    "time_signature_range": TS_MAPS,
    "use_programs": False,
    "use_sustain_pedals": False,
    "nb_tempos": 32,  # nb of tempo bins
    "tempo_range": (40, 250),  # (min, max)
}
config = miditok.TokenizerConfig(**TOKENIZER_PARAMS2)

# Creates the tokenizer
tokenizer = miditok.REMI(config)

In [6]:
tokenizer = pickle.load(open('tokenizer_bpe.p', 'rb'))

In [7]:
len(tokenizer.vocab_bpe.keys())

2000

## From raw MIDI

### Load the samples and transpose them to C major/ A minor

In [31]:
test_ts = []
for (dirpath, dirnames, filenames) in os.walk('test_toks'):
    test_ts += [os.path.join(dirpath, file) for file in filenames]

In [8]:
def transpose_midi(midi_file_path, out_dir):

    try:     
        # Load MIDI file
        midi_stream = converter.parse(midi_file_path)
    
        # Analyze key signature of the original MIDI file
        original_key = midi_stream.analyze('key')
        
        original_mode = original_key.mode  # Get the mode of the original key
    
        # Set the target key based on the mode of the original key
        if original_mode == 'major':
            new_key = key.Key('C')
        elif original_mode == 'minor':
            new_key = key.Key('a')
    
        # Calculate the interval to transpose
        interval = key.interval.Interval(original_key.tonic, new_key.tonic)
    
        # Transpose the entire stream
        transposed_stream = midi_stream.transpose(interval)

        if out_dir is not None:
            base_midi = os.path.basename(midi_file_path)
            output_path = os.path.join(out_dir, base_midi)
    
            transposed_stream.write('midi', fp=output_path)
        else:
            return transposed_stream
    except:
        pass

In [32]:
for midi_path in tqdm.tqdm(test_ts):
    transpose_midi(midi_path, 'test_toks_norm')

100%|██████████| 13/13 [00:16<00:00,  1.29s/it]


In [33]:
test_ts_norm = []
for (dirpath, dirnames, filenames) in os.walk('test_toks_norm'):
    test_ts_norm += [os.path.join(dirpath, file) for file in filenames]

In [14]:
transpose_midi('Piano.mid', 'test_toks_norm')

In [35]:
midi = tokenizer(test_ts_norm[0])
tokenizer.apply_bpe(midi)

### Generate many files

In [38]:
batch_size = 4
for f in tqdm.tqdm(test_ts_norm):
    midi = tokenizer(f)
    tokenizer.apply_bpe(midi)
    name = os.path.basename(f)
    name = name[:-4]
    tokens = ([x for x in midi[0]])
    batch = [tokens]*batch_size
    tokens = torch.LongTensor(batch).cuda()
    with autocast():
        sample = model.module.generate(tokens, 512, temperature=1, return_prime=False, verbose=False)
    
    for i in range(0, batch_size):
        out = sample[i].tolist()
        out = [x for x in out if x != 2001]
        out_midi = tokenizer.tokens_to_midi([out])
        out_midi.dump(f'generated_bpe/gen_{name}_{i}.mid')

100%|██████████| 13/13 [12:59<00:00, 59.95s/it]


### Generate from signle file

In [9]:
midi = tokenizer('test_toks_norm/Piano.mid')
tokenizer.apply_bpe(midi)

In [10]:
batch_size = 8
tokens = ([x for x in midi[0]])
batch = [tokens]*batch_size
tokens = torch.LongTensor(batch).cuda()
with autocast():
    sample = model.module.generate(tokens, 512, temperature=1, return_prime=False, verbose=True)

for i in range(0, batch_size):
    out = sample[i].tolist()
    out = [x for x in out if x != 2001]
    out_midi = tokenizer.tokens_to_midi([out])
    out_midi.dump(f'generated_custom/gen_Piano_mine_{i}.mid')

Generating sequence of max length: 512
0 / 512
32 / 512
64 / 512
96 / 512
128 / 512
160 / 512
192 / 512
224 / 512
256 / 512
288 / 512
320 / 512
352 / 512
384 / 512
416 / 512
448 / 512
480 / 512


## Generate from tokenized data

### Load tokenized sample

In [39]:
test_fs = []
for (dirpath, dirnames, filenames) in os.walk('data_remi_norm_bpe/test'):
    test_fs += [os.path.join(dirpath, file) for file in filenames]

In [40]:
test_fs 

['data_remi_norm_bpe/test/37fce7d256928de37af07d4785e308c2.json',
 'data_remi_norm_bpe/test/7a93a72818c38cdd83c7450a1a7763ff.json',
 'data_remi_norm_bpe/test/prelud24.json',
 'data_remi_norm_bpe/test/2f09ede0d2c1612152a5d4b7b1d056a2.json',
 'data_remi_norm_bpe/test/fe4125ffccdd6867e388836babcb9249.json',
 'data_remi_norm_bpe/test/7a8773bf7d216941dbdb72cc5b1953b6.json',
 'data_remi_norm_bpe/test/183f1b1d489c3bf1d9bed6e6877d88d6.json',
 'data_remi_norm_bpe/test/c566cd86984aa1eac375fa81d4e459c8.json',
 'data_remi_norm_bpe/test/fae0dce714f5999aaf70a235afac0299.json',
 'data_remi_norm_bpe/test/d42551845593660dec5cca0a51956303.json',
 'data_remi_norm_bpe/test/c4c7ed439995078374ac1f4a8bab85c6.json',
 'data_remi_norm_bpe/test/f61c5f0747f572202f53e52a343db6f4.json',
 'data_remi_norm_bpe/test/aca9c4afb4cca503dfec7bdee13f4b04.json',
 'data_remi_norm_bpe/test/898fe13aee487da2b9d9f648142e23cf.json',
 'data_remi_norm_bpe/test/14a94427536d12a213c2115f1620a8fb.json',
 'data_remi_norm_bpe/test/48cc235c

In [13]:
midi = [json.load(open(test_fs[2]))['ids'][0]]

### Single file generation

#### Load into batches

In [14]:
batch_size = 8

In [15]:
tokens = ([x for x in midi[0]])
batch = [tokens]*batch_size
tokens = torch.LongTensor(batch).cuda()

In [16]:
tokens

tensor([[370, 310,  27,  ..., 668,  51, 668],
        [370, 310,  27,  ..., 668,  51, 668],
        [370, 310,  27,  ..., 668,  51, 668],
        ...,
        [370, 310,  27,  ..., 668,  51, 668],
        [370, 310,  27,  ..., 668,  51, 668],
        [370, 310,  27,  ..., 668,  51, 668]], device='cuda:0')

In [17]:
with autocast():
    sample = model.module.generate(tokens, 512, temperature=1, return_prime=False)

Generating sequence of max length: 512
0 / 512
32 / 512
64 / 512
96 / 512
128 / 512
160 / 512
192 / 512
224 / 512
256 / 512
288 / 512
320 / 512
352 / 512
384 / 512
416 / 512
448 / 512
480 / 512


In [23]:
out = sample[0].tolist()
out = 
out2 = tokenizer.decode_bpe(out)

#### Generate in a loop

In [26]:
gen_list = list()
for i in range(0, batch_size):
    out = sample[i].tolist()
    out = [x for x in out if x != 2001]
    out_midi = tokenizer.tokens_to_midi([out])
    out_midi.dump(f'generated_bpe/generated_{i}.mid')

### Generate from many files

In [42]:
batch_size = 4
for f in tqdm.tqdm(test_fs[0:10]):
    midi = [json.load(open(f))['ids'][0]]
    name = os.path.basename(f)
    name = name[:-5]
    tokens = ([x for x in midi[0]])
    batch = [tokens]*batch_size
    tokens = torch.LongTensor(batch).cuda()
    with autocast():
        sample = model.module.generate(tokens, 512, temperature=1, return_prime=False, verbose=False)
    
    for i in range(0, batch_size):
        out = sample[i].tolist()
        out = [x for x in out if x != 2001]
        out_midi = tokenizer.tokens_to_midi([out])
        out_midi.dump(f'generated_bpe/gen_{name}_{i}.mid')

100%|██████████| 10/10 [10:04<00:00, 60.41s/it]


## Zip all generations

In [50]:
from zipfile import ZipFile

def zip_folder(folder_path, zip_path):
    with ZipFile(zip_path, 'w') as zip_file:
        for foldername, subfolders, filenames in os.walk(folder_path):
            for filename in filenames:
                file_path = os.path.join(foldername, filename)
                arcname = os.path.relpath(file_path, folder_path)
                zip_file.write(file_path, arcname)

# Example usage:
folder_to_zip = 'generated'
zip_file_path = 'gens.zip'

zip_folder(folder_to_zip, zip_file_path)

In [46]:
import random

def zip_random_files(folder_path, zip_path, sample_size):
    all_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path)]
    
    # Ensure the sample size is not greater than the number of files in the folder
    sample_size = min(sample_size, len(all_files))

    # Randomly select files for the sample
    selected_files = random.sample(all_files, sample_size)

    with ZipFile(zip_path, 'w') as zip_file:
        for file_path in selected_files:
            arcname = os.path.relpath(file_path, folder_path)
            zip_file.write(file_path, arcname)

# Example usage:
folder_to_sample = 'norm_midi'
zip_file_path = 'midi_sample.zip'
sample_size = 100  # Adjust this to the desired sample size

zip_random_files(folder_to_sample, zip_file_path, sample_size)